In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# import
cons = pd.read_excel('wine-consumption-per-person.xlsx')

prod = pd.read_excel('wine-production.xlsx')

un = pd.read_csv('UNSD — Methodology.csv', delimiter=';')
sub_un=un[['Region Name','Sub-region Name','ISO-alpha3 Code']]

lat_long = pd.read_csv('countries_codes_and_coordinates.csv')
sub_lat=lat_long[['Alpha-3 code','Latitude (average)', 'Longitude (average)']]
sub_lat['Latitude (average)']=sub_lat['Latitude (average)'].apply(lambda x: x[:-1][2:]).astype(float)
sub_lat['Longitude (average)']=sub_lat['Longitude (average)'].apply(lambda x: x[:-1][2:]).astype(float)
sub_lat['Alpha-3 code']=sub_lat['Alpha-3 code'].apply(lambda x: x[:-1][2:])

wdi=pd.read_csv('WDIData.csv').drop(['Country Name','Indicator Code'],axis=1)

hdi = pd.read_csv('human-development-index.csv')

print('\033[1mWine Consumption:')
display(cons.head(3))
print('\n')
print('\033[1mWine Production:')
display(prod.head(3))
print('\n')
print('\033[1mContinents & Sub-Regions:')
display(sub_un.head(3))
print('\n')
print('\033[1mLatitude & Longitude:')
display(sub_lat.head(3))
print('\n')
print('\033[1mWDI:')
display(wdi.head(3))
print('\n')
print('\033[1mHuman Development:')
display(hdi.head(3))

Wine Consumption:


,Entity,Code,Year,"Indicator:Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol) - Beverage Types:Wine"
0,Afghanistan,AFG,1961,0.0
1,Afghanistan,AFG,1962,0.0
2,Afghanistan,AFG,1963,0.0




Wine Production:


,Country,Code,Year,Production (tonnes),"Indicator:Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol) - Beverage Types:Wine"
0,Albania,ALB,2019,15925,1.15
1,Albania,ALB,2018,15290,1.12
2,Albania,ALB,2017,16868,1.11




Continents & Sub-Regions:


...

,Alpha-3 code,Latitude (average),Longitude (average)
0,AFG,33.0,65.0
1,ALB,41.0,20.0
2,DZA,28.0,3.0




WDI:


,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,AFE,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.936004,17.337896,17.687093,18.140971,18.491344,18.825520,19.272212,19.628009,NaN,NaN
1,AFE,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.499471,6.680066,6.859110,7.016238,7.180364,7.322294,7.517191,7.651598,NaN,NaN
2,AFE,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.855399,38.046781,38.326255,38.468426,38.670044,38.722783,38.927016,39.042839,NaN,NaN




Human Development:


,Entity,Code,Year,Human Development Index
0,Afghanistan,AFG,1990,0.273
1,Afghanistan,AFG,1991,0.279
2,Afghanistan,AFG,1992,0.287


In [3]:
def wdi_indicator_getter(indicator):   
    data = wdi.loc[wdi['Indicator Name'].isin([indicator])].reset_index().drop('index',axis=1)

    ind = pd.DataFrame({'ISO_Code':[0], 'Year':[0], 'Values':[0]})

    for i in range(1960,2020):
        new_ind = pd.DataFrame()
        new_ind['ISO_Code'] = 0
        new_ind['Year'] = 0
        new_ind['Values'] = 0
        for j in range(len(data)):
            new_ind.loc[j, 'ISO_Code'] = data['Country Code'].iloc[j]
            new_ind.loc[j, 'Year'] = i
            new_ind.loc[j, 'Values'] = data.loc[data['Country Code'].isin([data['Country Code'].iloc[j]])][str(i)].iloc[0]
        ind = ind.append(new_ind)

    ind.drop(0, axis=0, inplace=True)
    return ind

gdp_df = wdi.loc[

    # checking for words GDP and US independently of their order on the target string 
    (wdi['Indicator Name'].str.contains(r'(?=.*GDP)(?=.*US)', case=False))
    &
    # excluding words that would return irrelevant indicators
    (~ wdi['Indicator Name'].str.contains('constant|capita|oil|%|kg', case=False))]

# saving index in variable (Indicator Code)

gdp_df['Indicator Name'].unique()

In [ ]:
# data engineering
sub = prod[['Code', 'Year', 'Production (tonnes)']]
df = cons.merge(sub, on=['Code', 'Year'], how='left').fillna(0.)
df=df.merge(sub_un, left_on='Code', right_on='ISO-alpha3 Code', how='left').drop('ISO-alpha3 Code', axis=1).rename({'Region Name':'Region', 'Sub-region Name':'Sub_region', 'Code':'ISO_Code', 'Indicator:Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol) - Beverage Types:Wine':'Wine_Consumption_per_capita', 'Entity':'Country', 'Production (tonnes)':'Wine_Production_liters'}, axis=1)
df=df.merge(sub_lat, left_on='ISO_Code', right_on='Alpha-3 code', how='left').drop('Alpha-3 code', axis=1)
df['Wine_Production_liters'] = df['Wine_Production_liters'] *1000
pop = wdi_indicator_getter('Population, total')
df=df.merge(pop, on=['ISO_Code', 'Year'], how='left').rename({'Values':'Population'}, axis=1)
df['Total_Wine_Consumption'] = df['Wine_Consumption_per_capita'] * df['Population']
gdp = wdi_indicator_getter('GDP (current US$)').fillna(0.)
df=df.merge(gdp, on=['ISO_Code', 'Year'], how='left').rename({'Values':'GDP_USD', 'Latitude (average)':'Latitude', 'Longitude (average)':'Longitude'}, axis=1)

df.head()

In [ ]:
df.set_index('Country').to_csv('Dash_Macro.csv')

In [ ]:
# plot ideas

# evolution plot
# scatter
# maps
# 

In [ ]:
# annual consuption per capita * pop = total annual consumption (pure al)
# comparing GDP per capita, wine consumption (if rich/developed countries consume more wine in average)
#
#
#